In [ ]:
import datetime

class Appointment():
    
    def __init__(self, meeting_type, patient, doctor, timeframe):
        self.meeting_type = meeting_type
        self.patient = patient
        self.doctor = doctor
        self.timeframe = timeframe
    
    def __str__(self):
        return(f"Meeting type: {self.meeting_type} , Patient: {self.patient}, Doctor: {self.doctor}, Time: {self.timeframe[0]}-{self.timeframe[1]}")

In [ ]:
class Patient():

    def __init__(self, name = "Patient"):
        self.name = name
        self.appointments = []
        self.previous_appointments = []
        
    def __str__(self):
        return f"{self.name}"
    def add_appointment(self, appointment):
        self.appointments.append(appointment)
    
    def complete_appointment(self, appointment):
        if(appointment in self.appointments):
            self.previous_appointments.append(appointment)
            self.appointments.remove(appointment)
    
    def cancel_appointment(self, appointment):
        if(appointment in self.appointments):
            self.appointments.remove(appointment)
    
    def get_appointments(self):
        print(f"Appointment list for Patient: {self.name}")
        for app in self.appointments:
            print(app)

In [ ]:
class Doctor():
    def __init__(self, name = "Doctor"):
        self.name = name
        self.appointments = []
        self.unavailable = []
        self.previous_appointments = []
    
    def __str__(self):
        return f"{self.name}"
    def add_appointment(self, appointment):
        self.appointments.append(appointment)
    
    def complete_appointment(self, appointment):
        if(appointment in self.appointments):
            self.previous_appointments.append(appointment)
            self.appointments.remove(appointment)
    
    def cancel_appointment(self, appointment):
        if(appointment in self.appointments):
            self.appointments.remove(appointment)
            
    def get_appointments(self):
        print(f"Appointment list for Doctor: {self.name}")
        for app in self.appointments:
            print(app)

In [49]:
class Schedule():
    
    def __init__(self):
        self.sched = []
    
    def create_appointment(self,meeting_type, patient, doctor, timeframe):
        appointment = Appointment(meeting_type, patient, doctor, timeframe)
        patient.add_appointment(appointment)
        doctor.add_appointment(appointment)
        self.sched.append(appointment)
    
    def cancel_appointment(self, appointment):
        if(appointment in self.sched):
            appointment.patient.cancel_appointment(appointment)
            appointment.doctor.cancel_appointment(appointment)
            self.sched.remove(appointment)
    def get_appointments(self):
        print("List of appoints in the schedule: ")
        for app in self.sched:
            print(app)

In [50]:
patients = [Patient("Bob"), Patient("Tom"), Patient("John")]
doctors = [Doctor("Jim"), Doctor("Pam"), Doctor("Sam")]
sched = Schedule()

In [55]:
from IPython.display import clear_output
def main():
    while(True):
        begin = input("Would you like to schedule an appointment? ")
        if(begin == "No"):
            break
        choice = None
        while(True):
            choice = input("What type of appointment would you like: [Online] [Phone] [Office] ")
            if(choice == "Online" or choice == "Phone" or choice == "Office"):
                break
            clear_output()
            print("Not a valid option! ")
        while(True):
            person = input("Which patient is this for? ")
            try:
                patient = [p for p in patients if p.name == person]
                if(patient):
                    print(f"Patient Selected: {patient[0]}")
                    break
                else:
                    print("Patient not found! ")
            except:
                print("Patient not found! ")
        while(True):
            doc = input("Which doctor is this for? ")
            try:
                doctor = [d for d in doctors if d.name == doc]
                if(doctor):
                    print(f"Doctor selected: {doctor[0]}")
                    break
                else:
                    print("Doctor not found! ")
            except:
                print("Doctor not found! ")
        while(True):
            try:
                time = input('''What time would you like this appointment? Office appointments take an hour while phone and online appointments take half an hour. Please choose between 09:00 and 17:00 and increments of half hours! [HH:MM] ''')
                time = time.split(":")
                if(len(time) == 2):
                    start = datetime.time(int(time[0]), int(time[1]))
                    if(choice == "Office"):
                        endtime_h = int(time[0])+1
                        endtime_m = int(time[1])
                    else:
                        endtime_h = int(time[0])+round((int(time[1])+30)/60)
                        endtime_m = (int(time[1])+30)%60
                    end = datetime.time(int(time[0])+1, int(time[1])) if choice == "Office" else datetime.time(endtime_h,endtime_m)
                    break
            except:
                print("Incorrect format for time! ")
        sched.create_appointment(choice, patient[0], doctor[0], (start,end))
        print(f"Appointment created for {patient[0]} with {doctor[0]} at {choice} at time {start} - {end}")

main()

Would you like to schedule an appointment? O
What type of appointment would you like: [Online] [Phone] [Office] Online
Which patient is this for? O
Patient not found! 
Which patient is this for? Tom
Patient Selected: Tom
Which doctor is this for? Sam
Doctor selected: Sam
What time would you like this appointment? Office appointments take an hour while phone and online appointments take half an hour. Please choose between 09:00 and 17:00 and increments of half hours! [HH:MM] 14:00
Appointment created for Tom with Sam at Online at time 14:00:00 - 14:30:00
Would you like to schedule an appointment? No


In [56]:
sched.get_appointments()

List of appoints in the schedule: 
Meeting type: Office , Patient: Bob, Doctor: Jim, Time: 10:30:00-11:30:00
Meeting type: Phone , Patient: Tom, Doctor: Pam, Time: 13:00:00-13:30:00
Meeting type: Online , Patient: Tom, Doctor: Sam, Time: 14:00:00-14:30:00
